# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


---

In [28]:
import pandas as pd
import numpy as np
import yaml

In [29]:
with open("config.yaml", "r") as config:
    data_folder = yaml.safe_load(config)['data_folder']

# Load rows; skip first 64
df = pd.read_csv(data_folder + 'KNMI_20181231.txt.tsv', skiprows=63)

# Remove first empty row
df = df.drop(df.index[[0]])

# Rename the columns
df.rename(columns={'# STN': 'Station', 'YYYYMMDD': 'Date', '   TG': 'Tmean', '   TN': 'Tmin', '   TX': 'Tmax'}, inplace=True)

# Filter required columns
df = df[['Station', 'Date', 'Tmean', 'Tmin', 'Tmax']]

# Filter data for station 270
df = df.loc[df['Station'] == 270]

df.head()

/tmp/ipykernel_613303/2117724308.py:5: DtypeWarning: Columns (0,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder + 'KNMI_20181231.txt.tsv', skiprows=63)


,Station,Date,Tmean,Tmin,Tmax
97642,270,20000101.0,42,-4,79
97643,270,20000102.0,55,33,74
97644,270,20000103.0,74,49,89
97645,270,20000104.0,46,22,75
97646,270,20000105.0,41,14,56


<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [30]:
# Replace empty cells with NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Change station column type to int
df['Station'] = df['Station'].astype('int')

# Change temperature columns dtype to float
df[['Tmean', 'Tmin', 'Tmax']] = df[['Tmean', 'Tmin', 'Tmax']].apply(pd.to_numeric, errors='coerce')

# Change date column dtype to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Set the date as index and sort by the index
df = df.set_index('Date').sort_index()

# Change temperatures to celcius degrees
df[['Tmean', 'Tmin', 'Tmax']] = df[['Tmean', 'Tmin', 'Tmax']].div(10)

# Find unique leap years in the dataset
leap_years = df.loc[((df.index.month == 2) & (df.index.day == 29))].index.year.unique()

# Drop all the records where the year is in the leap_years list
df = df.drop(df[df.index.year.isin(leap_years)].index)

df.head()

,Station,Tmean,Tmin,Tmax
Date,,,,
2001-01-01,270,2.1,0.4,3.8
2001-01-02,270,6.3,3.7,8.5
2001-01-03,270,5.3,2.6,7.9
2001-01-04,270,6.5,4.6,7.9
2001-01-05,270,6.6,5.6,8.3


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [31]:
#Test your outcome
#write code to check if you have done the above

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5110 entries, 2001-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Station  5110 non-null   int64  
 1   Tmean    5110 non-null   float64
 2   Tmin     5110 non-null   float64
 3   Tmax     5110 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 199.6 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [32]:
def month_day(df_multipleyears):    
    # Split the dataframe
    before_2018 = df[:'2018-01-01']
    after_2018 = df['2018-01-01':]
    
    # Group data by month
    df_groupedbymonthday = df.groupby(by=[df.index.month.rename('Month'), df.index.day.rename('Day')]).min(['Tmin'])['Tmin']
    
    print(df_groupedbymonthday)

In [33]:
#Test your code
def test_reformed(df):
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day(df)

test_reformed(df)

Month  Day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -6.7
       5      -6.2
              ... 
12     27     -6.0
       28     -7.4
       29     -7.3
       30     -6.7
       31     -5.1
Name: Tmin, Length: 365, dtype: float64


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [34]:
#your code here

def moving_average(data, window):
    return [round((sum(data[i:i+window]) / window), 2) for i in range(len(data))]


def exponential_smoothing(data, alpha):
    predictions = [data[0]]

    for i in range(1, len(data)):
        prediction = alpha * data[i - 1] + (1 - alpha) * predictions[i - 1]
        predictions.append(round(prediction, 2))
    
    return predictions
        

def smooth_data(data, method = 'moving_average', window = 3, alpha = 0.3):
    if method == 'moving_average':
        return moving_average(data, window)
    elif method == 'exponential_smoothing':
        return exponential_smoothing(data, alpha)
    else:
        print('Method must either be moving_average or exponential_smoothing')


#your motivation here

'''
There are a lot of smoothing algorithms available, so it was a bit hard to make a choice.
I used the list of algorithms with their descriptions on Wikipedia to make a choice:
https://en.wikipedia.org/wiki/Smoothing

Knowing our data is time based, I selected the following two algorithms based on their descriptions and pros:

1. Exponential smoothing
2. Moving average

I think that the moving average meets our demands as best, since it 'has been adjusted to allow for seasonal or 
cyclical components of a time series', according to Wikipedia. Because we work with temperature data over time,
the seasonal aspect of the moving average is very applicable to our data.
'''

"\nThere are a lot of smoothing algorithms available, so it was a bit hard to make a choice.\nI used the list of algorithms with their descriptions on Wikipedia to make a choice:\nhttps://en.wikipedia.org/wiki/Smoothing\n\nKnowing our data is time based, I selected the following two algorithms based on their descriptions and pros:\n\n1. Exponential smoothing\n2. Moving average\n\nI think that the moving average meets our demands as best, since it 'has been adjusted to allow for seasonal or \ncyclical components of a time series', according to Wikipedia. Because we work with temperature data over time,\nthe seasonal aspect of the moving average is very applicable to our data.\n"

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



In [35]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Band, ColumnDataSource, DatetimeTickFormatter
output_notebook()

Loading BokehJS ...

In [36]:
# Get values of 2018
df_2018 = df[df.index.year == 2018]

# Get min and max values of all data
df_all = df.groupby(by=[df.index.month.rename('Month'), df.index.day.rename('Day')]).agg({
    'Tmin': 'min',
    'Tmax': 'max'
})

# Reset indices of the dataframes
df_all = df_all.reset_index(drop=True)
df_2018 = df_2018.reset_index(drop=True)

# Create the figure
p = figure(
    title="Mean temperature 2018",
    width=900, y_range=(df_all['Tmin'].min() - 3, df_all['Tmax'].max() + 3),
    x_range=(1, 365),
    x_axis_label="Time in days of the year",
    y_axis_label="Temperature in degrees Celsius"
)

# Create mean line
p.line(df_2018.index, df_2018['Tmean'], legend_label="Mean temperature", color="blue", line_width=2)

# Create column data source object to pass to the Band constructor
source = ColumnDataSource(df_all)

# Create Band object and add it to layout
band = Band(base='index', lower="Tmin", upper="Tmax", source=source, fill_alpha=0.3, fill_color="gray", line_color="black")
p.add_layout(band)

show(p)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [37]:
import hvplot.pandas
import param
import panel as pn
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [38]:
# Create widgets
smoothing_select = pn.widgets.Select(name='Distribution', width=150, options=['Discrete', 'Moving average', 'Exponential smoothing'])
movav_slider = pn.widgets.IntSlider(name='Integer Slider', start=0, end=100, value=3, width=150)
exponential_slider = pn.widgets.FloatSlider(name='Float Slider', width=150, start=0, end=1, step=0.01, value=0.3)

@pn.depends(smoothing_select.param.value, movav_slider.param.value, exponential_slider.param.value)
def draw_plot(select_value, movav_value, exponential_value):
    # Create the figure
    p = figure(
        title="Mean temperature 2018",
        width=800, y_range=(df_all['Tmin'].min() - 3, df_all['Tmax'].max() + 3),
        x_range=(1, 365),
        x_axis_label="Time in days of the year",
        y_axis_label="Temperature in degrees Celsius"
    )

    y_data = df_2018['Tmean']

    if select_value == 'Discrete':
        y_data = df_2018['Tmean']
        movav_slider.visible = False
        exponential_slider.visible = False
    if select_value == 'Moving average':
        y_data = smooth_data(y_data, window=movav_value)
        movav_slider.visible = True
        exponential_slider.visible = False
    if select_value == 'Exponential smoothing':
        y_data = smooth_data(y_data, method='exponential_smoothing', alpha=exponential_value)
        movav_slider.visible = False
        exponential_slider.visible = True
    
    # Create mean line
    p.line(df_2018.index, y_data, legend_label="Mean temperature", color="blue", line_width=2)

    # Create column data source object to pass to the Band constructor
    source = ColumnDataSource(df_all)

    # Create Band object and add it to layout
    band = Band(base='index', lower="Tmin", upper="Tmax", source=source, fill_alpha=0.3, fill_color="gray", line_color="black")
    p.add_layout(band)

    return p



layout = pn.Row(
    pn.Column(
        smoothing_select,
        movav_slider,
        exponential_slider,
    ),
    draw_plot
)

layout

Row
    [0] Column
        [0] Select(name='Distribution', options=['Discrete', 'Moving avera...], value='Discrete', width=150)
        [1] IntSlider(end=100, name='Integer Slider', value=3, visible=False, width=150)
        [2] FloatSlider(name='Float Slider', step=0.01, value=0.3, visible=False, width=150)
    [1] ParamFunction(function)